In [1]:
import numpy as np
import pandas as pd

In [2]:
players = pd.read_csv('../dataset light memory/players_fifa22.csv')
teams = pd.read_csv('../dataset light memory/teams_fifa22.csv')

In [ ]:
list(players.columns)

In [ ]:
import random
from collections import deque
import os
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
        
class DQNetwork:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen = 2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.01
        self.model = self._build_model()
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim = self.state_size, activation = 'relu'))
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(self.action_size, activation = 'linear'))
        model.compile(loss='mse', optimizer = Adam(learning_rate = self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in batch:
            target = reward #target = 80
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0])) 
            target_cur = self.model.predict(state)
            target_cur[0][action] = target
            self.model.fit(state, target_cur, epochs = 1, verbose = 0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    def load(self, name):
        self.model.load_weights(name)
    def save(self, name):
        self.model.save_weights(name)
    


In [4]:
import gym

env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
n_episodes = 1001

In [6]:
arr = env.reset()

In [55]:
done = False
env.reset()
while not done:
    a, b , done, _ = env.step(0)
    print(a,b,done)

[-0.00625125 -0.22040966  0.03027815  0.29822242] 1.0 False
[-0.01065945 -0.41594985  0.0362426   0.60029864] 1.0 False
[-0.01897844 -0.61155957  0.04824857  0.90417355] 1.0 False
[-0.03120963 -0.8073006   0.06633205  1.2116231 ] 1.0 False
[-0.04735565 -1.0032132   0.0905645   1.5243329 ] 1.0 False
[-0.06741991 -1.1993045   0.12105117  1.8438541 ] 1.0 False
[-0.091406   -1.3955357   0.15792824  2.171549  ] 1.0 False
[-0.11931671 -1.5918063   0.20135923  2.5085294 ] 1.0 False
[-0.15115283 -1.7879367   0.2515298   2.8555813 ] 1.0 True


In [ ]:
# agent = DQNetwork(state_size, action_size)

# for e in range(n_episodes):
#     state = env.reset()
#     state = np.reshape(state, [1, state_size])
#     for time in range(5000):
#         #env.render()
#         action = agent.act(state)
#         next_state, reward, done, _ = env.step(action)
#         reward = reward if not done else -10
#         next_state = np.reshape(next_state, [1, state_size])
#         agent.remember(state, action, reward, next_state, done)
#         state = next_state
#         if done:
#             print(f"episode: {e}/{n_episodes}, score: {time}, e: {round(agent.epsilon, 2)}")
#             break
#     if len(agent.memory) > batch_size:
#         agent.replay(batch_size)

In [ ]:
state = env.reset()
state = np.reshape(state, [1, state_size])
next_state, reward, done, _ = env.step(action)

In [ ]:
testdqn.model.predict([[1,2,3,4,5,6,7,8,9,10]])

## env logic

#### env reset()
#### env.step()
#### env.

In [ ]:
class Player:
    def __init__(self, name = '', position = '', position_rating = 0, value = 0, wage = 0):
        self.name = name
        #self.overall_rating = overall_rating
        self.position = position
        self.position_rating = position_rating
        self.value = value
        #self.wage = wage

In [ ]:
class Environment:
    def __init__(self, players, money):
        self.players = players[['ID', 'Name', 'ValueEUR', 'Overall', 'STRating', 'LWRating', 'LFRating', 'CFRating', 'RFRating', 'RWRating',
                                'CAMRating', 'LMRating', 'CMRating', 'RMRating', 'LWBRating',
                                'CDMRating', 'RWBRating', 'LBRating', 'CBRating', 'RBRating', 'GKRating']]
        self.formation = ['GK', 
                         'LB', 'CB', 'CB', 'RB', 
                         'LM', 'CAM', 'RM',
                         'LW', 'ST', 'RW']
        self.reward = 0
    def reset(self):
        self.money = np.random.randint(10000000, 1000000000, size = 1)
        self.formation = ['GK', 
                         'LB', 'CB', 'CB', 'RB', 
                         'LM', 'CAM', 'RM',
                         'LW', 'ST', 'RW']
        self.posid = 0
        self.done = False
        self.squad = []
        self.squad_players_ids = []
        state = np.array([self.money, self.posid, self.squad_players_ids])
        return state
    
    def step(self, action):
        money_exceeded_flag = False
        if action in self.squad_players_ids:
            return [[self.money, self.posid, self.squad_players_ids], -10, False, {}]
        
        if len(self.squad) + 1 == 11:
            self.done = True
        self.squad_players_ids.append(action)
        player = Player(players.iloc[action]['Name'], 
                                 self.formation[self.posid],
                                 players.iloc[action][f'{self.formation[self.posid]}Rating'],
                                 players.iloc[action]['ValueEUR'])
        self.squad.append(player)
        self.posid += 1
        self.money -= player.value
        if self.money < 0:
            money_exceeded_flag = True
        next_state = [self.money, self.posid, self.squad_players_ids]
        
        reward = self.evalSquad()
        if money_exceed_flag = True:
            done = True
            reward = -100
        if repeat_players_flag:
        return [next_state, reward, done, {}]
    
    def evalSquad(self):
        sum_rating = 0
        for player in self.squad:
            sum_rating += player.position_rating
        if len(self.squad) == 11:
            return sum_rating
        return round(sum_rating/len(squad), 2)

In [ ]:
env = Environment()

In [ ]:
for player in env.squad:
    print(f'name: {player.name}, position: {player.position}, Position rating: {player.position_rating}')

In [ ]:
print(env.evalSquad())